In [1]:
import numpy as np

## Ball to Handler

In [2]:
def length(p1, p2, axis=-1):
    return np.sqrt((p1[...,0]-p2[...,0])**2+(p1[...,1]-p2[...,1])**2)

MODELS = [
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P_4Res/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_78k/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_98k/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_166k/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P_messy/valid_results_205k/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy'
]

In [3]:
for m_path in MODELS:
    if m_path == '/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy':
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/rfeat_valid.npy'
    else:
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/feat_valid.npy'
    rec_data = np.load(m_path)
    rec_data = np.reshape(rec_data, [rec_data.shape[0], rec_data.shape[1], 11, 2])
    num_data,num_seq,num_player,num_feat = rec_data.shape
    feat_data = np.load(feat_path)[:,:,:5] # [?,?,6] -> [?,?,5] remove basket feature
    # mask to filter out ball passing  
    mask = np.sum(feat_data, axis=-1)!=0
    feat_data = np.argmax(feat_data, axis=-1) + 1 # ball:0, offense:1-5, defense:6-10
    # get index
    idx = np.repeat(np.repeat(np.arange(num_data), [num_seq]),[2])
    t_idx = np.repeat(np.tile(np.arange(num_seq), [num_data]),[2])
    feat_idx = np.repeat(np.reshape(feat_data, [-1,]),[2])
    xy_idx = np.tile(np.arange(2), [num_data*num_seq])
    # gathering
    handler_pos = rec_data[idx,t_idx,feat_idx,xy_idx].reshape([num_data, num_seq, num_feat])
    ball_pos = rec_data[:,:,0,:]
    # compute distance
    lens = length(handler_pos, ball_pos)
    dist = np.sum(lens*mask)/np.count_nonzero(mask)
    std = (np.sum((lens-dist)**2*mask)/(np.count_nonzero(mask)-1))**0.5
    print('{0:100}:  {1:.4f}, std:{2:.4f}'.format(m_path, dist, std))

/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy                       :  1.8084, std:1.2922
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy                               :  1.5991, std:1.1149
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy                             :  1.3632, std:0.9749
/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy                                       :  1.2063, std:1.0166
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P_4Res/results/reconstruct.npy                        :  1.5335, std:1.0931
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_78k/reconstruct.npy                   :  1.5303, std:1.1148
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_98k/reconstruct.npy                   :  1.5374, std:1.0986
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_166k/reconstruct.npy                  :  1.7312, std:1.1312
/workspace/data/nctu_cgv

## Ball to Defender

In [4]:
for m_path in MODELS:
    if m_path == '/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy':
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/rfeat_valid.npy'
    else:
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/feat_valid.npy'   
    rec_data = np.load(m_path)
    rec_data = np.reshape(rec_data, [rec_data.shape[0], rec_data.shape[1], 11, 2])
    num_data,num_seq,num_player,num_feat = rec_data.shape
    feat_data = np.load(feat_path)[:,:,:5] # [?,?,6] -> [?,?,5] remove basket feature
    # mask to filter out ball passing  
    mask = np.sum(feat_data, axis=-1)!=0
    defender_pos = rec_data[:,:,6:11,:]
    ball_pos = rec_data[:,:,0:1,:]
    # compute distance
    def_5lens = length(defender_pos, ball_pos)
    def_best = np.amin(def_5lens, axis=-1)
    dist =  np.sum(def_best*mask)/np.count_nonzero(mask)
    std = (np.sum((def_best-dist)**2*mask)/(np.count_nonzero(mask)-1))**0.5
    print('{0:100}:  {1:.4f}, std:{2:.4f}'.format(m_path, dist, std))

/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy                       :  6.7745, std:3.3128
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy                               :  6.8084, std:3.2307
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy                             :  6.5078, std:3.2255
/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy                                       :  6.1882, std:2.2477
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P_4Res/results/reconstruct.npy                        :  6.6968, std:3.0402
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_78k/reconstruct.npy                   :  6.9836, std:3.2482
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_98k/reconstruct.npy                   :  6.8304, std:3.4219
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_166k/reconstruct.npy                  :  6.9175, std:3.1302
/workspace/data/nctu_cgv

## Match Seq and Offense

In [36]:
sketch_data = np.load('/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_98k/seq_valid.npy').reshape([-1,50,6,2])
print(sketch_data.shape)
for m_path in MODELS:
    rec_data = np.load(m_path)
    rec_data = np.reshape(rec_data, [rec_data.shape[0], rec_data.shape[1], 11, 2])[:,:,0:6,:]
    num_data,num_seq,num_player,num_feat = rec_data.shape
    min_list = []
    for p in range(num_player):
        for t in range(num_seq):
            sk2rec = length(rec_data[:,:,p],sketch_data[:,t:t+1,p])
            min_ = np.amin(sk2rec, axis=-1)
            min_list.append(min_)
    mean_ = np.mean(np.array(min_list))
    print('{0:100}:  {1:.4f}'.format(m_path, mean_))

(1405, 50, 6, 2)
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy                       :  0.9708
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy                               :  0.9680
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy                             :  0.9710
/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy                                       :  0.7202
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P_4Res/results/reconstruct.npy                        :  1.0075
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_78k/reconstruct.npy                   :  1.1067
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_98k/reconstruct.npy                   :  1.0175
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D4P/valid_results_166k/reconstruct.npy                  :  0.9912
/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy                                             :  